In [4]:
import os
import csv
import pandas as pd
import numpy as np
import sklearn
import string
import statsmodels.api as sm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from nltk import tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [9]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [10]:
def display_topics(model, feature_names, num_topics, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        if topic_idx < num_topics:
            print("{:11}".format("Topic %d:" %(topic_idx)), end='')
            print(", ".join(['{:04.3f}*'.format(topic[i])+feature_names[i] \
                             for i in topic.argsort()[:-no_top_words-1:-1]]))

In [4]:
# Read in Data
data = pd.read_csv('vegas.csv')
data['userid'] = data['Unnamed: 0']
data.head()

C:\Users\xinro\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,business_id,name,city,state,postal_code,review_count,stars_x,categories,cool,date,funny,review_id,stars_y,text,useful,user_id,userid
0,0,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,Las Vegas,NV,89119.0,40.0,4.0,"Restaurants, Italian",0.0,2011-06-29 02:55:07,0.0,klcF45wKIOpJW_BhJslOJg,5.0,"We went there for dinner the other night, bein...",1.0,-Yz2wIcsdJxUOFMbTgoKQA,0
1,1,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,Las Vegas,NV,89119.0,40.0,4.0,"Restaurants, Italian",0.0,2010-10-06 18:20:13,0.0,Li-pQG6A7p5gbgZHTMeDSQ,4.0,i had the best Chicken Marcela ever. The spagh...,1.0,jYcf_e5p0UG0S-9gJq_tNA,1
2,2,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,Las Vegas,NV,89119.0,40.0,4.0,"Restaurants, Italian",0.0,2010-01-13 00:35:45,0.0,iRLX3dJ3ONvncIxPnXy1cw,5.0,Basically the best Italian in town for the pri...,1.0,nQC0JiPIk_jCooRDxpuw5A,2
3,3,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,Las Vegas,NV,89119.0,40.0,4.0,"Restaurants, Italian",0.0,2008-08-23 20:30:33,0.0,rklteWf9xnTU3fAtMFBRRw,3.0,Mmmmm delicious food and a little history. Mr....,1.0,Gv_-mtOKhWFtCjn9xFe0SQ,3
4,4,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,Las Vegas,NV,89119.0,40.0,4.0,"Restaurants, Italian",0.0,2009-06-01 20:02:55,0.0,UfRqM0RGdZa86hFcFEAnjw,3.0,"This is old Vegas, this atmosphere is old scho...",1.0,pabMYegF28KjHQ5hybAJ0A,4


In [5]:
data = data[data['postal_code'] == 89103].reset_index(drop = True)

In [9]:
len(data)

60974

In [26]:
# Split reviews into individual sentences 
df = pd.DataFrame(columns=['userid','sentence','stars'])
for i in range(50000,60974,1):
    sentences = tokenize.sent_tokenize(data.text[i])
    for j in sentences:
        df = df.append({'userid':data.userid[i],'sentence':j,'stars':data.stars_y[i]},ignore_index=True)

In [27]:
# Create Corpus for TFIDF
corpus_6 = []
for i in df.sentence:
        corpus_6.append(i)

In [18]:
df_6 = pd.read_csv('89103_6.csv')

In [7]:
corpus = []
for i in df.sentence:
        corpus.append(i)

In [15]:
#2
df.to_csv('89103_2.csv')
#corpus_2

In [19]:
#3
df.to_csv('89103_3.csv')
#corpus_3

In [22]:
#4
df.to_csv('89103_4.csv')
#corpus_4

In [25]:
#5
df.to_csv('89103_5.csv')
#corpus_5

In [28]:
#6
df.to_csv('89103_6.csv')
#corpus_6

In [20]:
# Create Corpus for TFIDF
corpus = []
corpus = corpus_1+corpus_2+corpus_3+corpus_4+corpus_5+corpus_6

In [11]:
n_components = 7
n_top_words = 15

# TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(corpus)

# NMF reduction
nmf = NMF(n_components=n_components).fit(tfidf)
W_pos = nmf.fit_transform(tfidf)

# Output Topics
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: great experience atmosphere prices price overall staff selection view sushi meal deal value flavor music
Topic #1: good really pretty price overall prices flavor thing experience selection quality chicken taste portions tasted
Topic #2: place love recommend amazing try awesome highly clean stars favorite eat sushi new nice looking
Topic #3: food delicious amazing fresh quality excellent fast authentic chinese mexican awesome ok price tasty thai
Topic #4: service friendly excellent customer fast staff attentive slow nice amazing bad horrible quick awesome terrible
Topic #5: time vegas just best like really ve restaurant buffet don order eat las came didn
Topic #6: definitely come recommend try coming ll worth highly return vegas visit going town wait soon



- Topic #0: Overall experience
- Topic #1: Price
- Topic #2: "Worth it"
- Topic #3: Food
- Topic #4: Service
- Topic #5: 
- Topic #6: "Worth it"

## Label Sentences

In [5]:
import glob
path =r'C:\Users\xinro\Downloads\89103' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)
df = frame

In [12]:
# Append Topic with highest score
array = []
# For all NMF array
for i in range(0,len(W_pos),1):
    # Create dictionary with Topics and its NMF scores for each sentence
    topic_dict = {}
    # Drop sentences that have length less than 10 by setting topic to -1
    if len(corpus[i])>=10:
        for ind, w in enumerate(W_pos[i]):
            topic_dict[ind] = w
        # Classify sentence to the topic with highest score
        array.append(max(topic_dict, key=topic_dict.get))
    else:
        array.append(-1)
# Create new column in df for topic
df['Topic'] = array

## Veder Sentiment Analysis

In [13]:
# Initialize Sentiment Intensity Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [14]:
# Append Sentiment Intensity Scores for each sentence
array = []
for i in df.sentence:
    # Generate Sentiment Intensity Scores and store in array
    score = analyser.polarity_scores(i)
    array.append(score['compound'])
# Create new column in df for sentiment intensity score
df['sentiment'] = array

In [15]:
# Initialize Final df of intensity scores
df_scores = pd.DataFrame(columns=['userid','0','1','2','3','4','5','6','stars'])
# For every user aggregate the sentiment scores by topic
for i in df.userid.unique():
    # Create df of scores from same user
    temp_df = df[df.userid==i].reset_index(drop=True)
    # For every topic
    topic_score = []
    for j in range(0,7,1):
        score = 0
        count = 0
        for k in range(0,len(temp_df),1):
            # If topic equal to current topic
            if temp_df.Topic[k] == j:
                # Add sentiment score
                score = score + temp_df.sentiment[k]
                # Increase count
                count = count + 1
        # If count = 0 then no score for topic
        if count==0:
            topic_score.append(0)
        # Else append average score for topic
        else:
            topic_score.append(score/count)
    # Insert UserId and Star Rating 
    topic_score.insert(0,temp_df.userid[0])
    topic_score.insert(len(topic_score),temp_df.stars[0])
    # Transform and Append into main df
    temp = pd.DataFrame(pd.Series(topic_score))
    temp = temp.transpose()
    temp.columns = df_scores.columns
    df_scores = df_scores.append(temp,ignore_index=True)

In [16]:
df_scores.head()

,userid,0,1,2,3,4,5,6,stars
0,1248.0,0.8779,0.0000,0.00000,0.0000,0.6249,0.22350,0.0000,5.0
1,1249.0,0.7579,0.4663,0.72705,0.0000,0.8271,0.64650,0.4201,5.0
2,1250.0,0.0000,0.0000,0.00000,0.0000,0.0000,0.15434,0.0000,5.0
3,1251.0,0.0000,0.0000,0.00000,0.8655,0.0000,0.50685,0.0000,5.0
4,1252.0,0.0000,0.0000,0.12530,0.0000,0.8074,0.49390,0.0000,5.0


##  Linear Regression

In [32]:
# Split into predictors and target
X = df_scores.drop(['userid','stars'],axis=1)
X = X.astype(float)
y = df_scores.stars
y = y.astype(float)
# Split Train vs Test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=1)
# Split Test set into validation & test set
X_test2, X_val, y_test2, y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test,random_state=1)

####  Simple linear regression

In [33]:
model = sm.OLS(y_train,sm.add_constant(X_train)).fit()
print(model.params)
print()
print('Mean Squared Error: ',mean_squared_error(y_val,model.predict(sm.add_constant(X_val))))
print('AIC: ',model.aic)

const    2.761842
0        0.673569
1        0.488325
2        0.823065
3        1.055124
4        0.996302
5        1.825861
6        0.674795
dtype: float64

Mean Squared Error:  1.1923921339959864
AIC:  147812.53528216275


In [ ]:
- Topic #0: Overall experience
- Topic #1: Price
- Topic #2: "Worth it"
- Topic #3: Food
- Topic #4: Service
- Topic #5: 
- Topic #6: "Worth it"

### 𝑦ℎ𝑎𝑡=2.76+0.67∗Overall+0.49∗Price+0.82∗𝑊𝑜𝑟𝑡ℎ+1.06∗𝐹𝑜𝑜𝑑+1.00∗Service+1.83∗Topic5+0.67∗Worth

#### Removed Intercept and Non-Topics

In [34]:
# Split into predictors and target
X = df_scores.drop(['userid','stars','4'],axis=1)
X = X.astype(float)
y = df_scores.stars
y = y.astype(float)
# Split Train vs Test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=1)
# Split Test set into validation & test set
X_test2, X_val, y_test2, y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test,random_state=1)

model = sm.OLS(y_train,X_train).fit()
print(model.params)
print()
print('Mean Squared Error: ',mean_squared_error(y_val,model.predict(X_val)))
print('AIC: ',model.aic)

0    2.232742
1    2.921851
2    2.292232
3    2.706688
5    4.302188
6    2.215236
dtype: float64

Mean Squared Error:  4.961835852693974
AIC:  215712.09669405263
